## The Knapsack Problem

This notebook contains different algorithms to solve the knapsack problem. The knapsack problem is defined as follows:

- let there be $n$ items denoted by $i = 0,..., n-1$
- let $v_i$ be value of an item and the $w_i$ weight
- let $K$ be the capacity of the knapsack
- let $x_i$ be a variable that takes the valie $1$ if item $i$ is taken in the knapsack and $0$ otherwise

The goal is to find the $k$ items to take in the knapsack that maximise the total value. The problem is formalised as: 
<br>
<br>
<br>
$$\begin{alignat}{3}
\text{maximise: }                &\quad&  \sum_{i \in 0, ..., n-1} v_i \cdot x_i  &&          & \\
\text{subject to: } &\quad&  \sum_{i \in 0, ..., n-1} w_i \cdot x_i  && \leq K   &\\
                    &\quad&  x_i \in {\{0,1\}}                       &&    &\quad \forall i=0,...,n-1
\end{alignat}\\
$$
<br>
<br>

The algorithms implemented to solve the problem are:
- greedy algorithm 
- dynamic program
- branch and bound (in progress)

In [20]:
from knapsack.problem_setup import parse_input_data, output_solution
from knapsack.dynamic_program import dp_solver
from knapsack.greedy import greedy_solver
from knapsack.branch_and_bound import BranchAndBound

Different knapsack files from the *data* folder can be entered in the cell below to test the algorithms on different problems. 

In [21]:
with open('data\ks_200_0', 'r') as input_data_file:
    input_data = input_data_file.read()
    
items, capacity = parse_input_data(input_data)

print("The knapsack has a capacity of {:,.0f} and there are {:,.0f} items to be considerd".format(capacity, len(items)))

The knapsack has a capacity of 100,000 and there are 200 items to be considerd


### Greedy Algorithm

In most cases, greedy algorithms are unable achieve optimality. However, they are useful for quickly finding a solution that can be used as a baseline with which to compare solutions from more complex algorithms; here is no exception.

The greediness of this algorithm is in relation to the value to weight ration of each item. That is, starting with the item that has the most value per weight units, items are taken progressively until the knapsack's capacity is filled.

Due to its simplicity, the algorithm is able to very quickly arrive at a feasible solution, even for very large problems, but without any guarantee of optimality.

In [22]:
decision_variables_gr, obj_value_gr = greedy_solver(items, capacity)
selected_items = [idx for idx, var in enumerate(decision_variables_gr) if var == 1]

In [23]:
print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_gr))
for item in selected_items:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))

The knapsack has a total value of 100,062 and contains:
- item 168 with weight 30,834 and value 30,918
- item 196 with weight 34,348 and value 34,446
- item 198 with weight 34,599 and value 34,698


### Dynamic Program

The dynamic program implementation for the knapsack problem is guaranteed to find an exact solution. However, with its "pseudo-polynomial" time complexity of $O(nK)$ it can perform poorly for problems with large number of potential items and/or a large capacity.  

This can be seen with the problems in the *data* folder. The dynamic program quickly solves small to medium sized problems such as the instance with 1,000 potential items and a capacity of 100,000. But for larger problems, such as the one with 10,000 items and a capacity of 1,000,000, the time complexity reaches a level where an optimal solution cannot be found within a practical period of time. Thus, a computationally more efficient alternative is needed for such problems.

In [24]:
decision_variables_dp, obj_value_dp = dp_solver(items, capacity)
selected_items = [idx for idx, var in enumerate(decision_variables_dp) if var == 1]

In [25]:
print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_dp))
for item in selected_items:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))

The knapsack has a total value of 100,236 and contains:
- item 118 with weight 24,559 and value 24,618
- item 120 with weight 24,810 and value 24,870
- item 122 with weight 25,061 and value 25,122
- item 126 with weight 25,563 and value 25,626


### Branch and Bound (in progress)

In [ ]:
bnb = BranchAndBound(input_data)
decision_variables_bnb, obj_value_bnb = bnb.solve()
selected_items_bnb = [idx for idx, var in enumerate(decision_variables_bnb) if var == 1]

In [ ]:
print("The knapsack has a total value of {:,.0f} and contains:".format(obj_value_dp))
for item in selected_items_bnb:
    print("- item {:,.0f} with weight {:,.0f} and value {:,.0f}".format(item, items[item].weight, items[item].value))